# Read epub file using UnstructuredEPubLoader and apply Underwoods list of ocr corrections

In [ ]:
#! pip install pypandoc

In [9]:
print ('import libraries')
# import libraries
from langchain_community.document_loaders import UnstructuredEPubLoader
from langchain.document_loaders import PyPDFLoader
import pandas as pd
import re
import os
import csv
import json
from tqdm import tqdm  # Import tqdm for progress tracking
import time

# Save start time
print ('start time')
start_time = time.time()
#######################

# Move to epub folder
os.chdir(r'.\literature epub')
# load text from epub
epub_file = '1860 A Residence in Jutland the Danish Isles and Copenhagen vol II.epub'
book_title = epub_file[:-5]
print (f'Start processing "{book_title}".\nWait a few seconds.')
loader = UnstructuredEPubLoader(epub_file, mode="elements")
epub_pages = loader.load()

# Extract data from epub and send it to a dataframe
content = []
source = []
element_no = []


# add counter to element number
count = 1 
for i in epub_pages:
       
    # return the text content, source, and element no.
    pageContent = i.page_content
    pageSource = i.metadata['source'][:-5]
    elementNo = count
    
    content.append(pageContent)
    source.append(pageSource)
    element_no.append(elementNo)

    count += 1
    
df = pd.DataFrame({'content': content, 'source': source, 'element_no': element_no})

######################
print ('Preprocess text')
# preprocess text
def preproces_text(text_string):
    # Store text data in variable
    # Remove newlines, replace apostrophe s with s, remove quotation marks
    text_string_replace = text_string.replace(r' \xad\n', '').replace(r'\xad\n', '') \
        .replace(r'\xad', '').replace(r'\xad ', '') \
        .replace(r'-\n', '').replace('—', ' ').replace(r'\n', ' ') \
        .replace("'s", "s").replace('"', ' ').replace("'", " ") \
        .replace('.', ' . ').replace(' .', ' . ').replace(',', ' ,') \
        .replace('!', ' !').replace('?', ' ?')

    # scrub for multiple signs
    # but keep full stops, commas, exclamation, and question marks
    text_string_re = re.sub(r':|;||||\||\+|\"|‘|’|“|”|\’|…|\-|–|—|\$|&|\*|>|<|\/||||»|«', '', text_string_replace)

    # Remove numbers and words containing numbers
    text_string_no_numbers = re.sub(r'\b\w*\d\w*\b', ' ', text_string_re)
    # Remove multiple whitespaces
    text_string_sub_additional_white_space = re.sub(r'\s+', ' ', text_string_no_numbers )

    return text_string_sub_additional_white_space

df['preprocessed_content'] = df['content'].apply(lambda x : preproces_text(x))

################

# go back to main folder
os.chdir('..')


print('Starting ocr correction - loading correction rules')
# Read the correction rules from the file
with open('Underwoods_CorrectionRules.txt', 'r', encoding='utf-8') as f:
    CorrectionRules_string = f.read()

# Process the correction rules
CorrectionRules_list = CorrectionRules_string.split('\n')
list_of_CorrectionRules_list = [i.split('\t') for i in CorrectionRules_list]
better_list_of_CorrectionRules = [i[0:2] for i in list_of_CorrectionRules_list]

# Add additional pairs
observations_to_add = [["Bruffels", 'Brussels'], ["fix", 'six'], ['Elsinore', 'Helsingør']]
better_list_of_CorrectionRules.extend(observations_to_add)

# Build correction dictionary
correction_dict = dict(better_list_of_CorrectionRules)



print('Ocr correction')

def ocr_post_correction(text):
    # Prepare for correction
    ### Keep track of the conditions of the words (capital, title, upper,lower)
    tokens = text.split()

    # Track the original case of each word
    original_case = [i for i in tokens]
    clean_text_list = [i.lower() for i in tokens]

    # Initialize the list to store corrected words and a dictionary to track corrections
    new_word_list = []
    corrections_tracker = {}

    # Iterate through the clean_text_list word list
    for index, word in enumerate(clean_text_list):
        # Check if the word has a correction in the dictionary
        if word in correction_dict:
            # If it does, append the corrected word to the new list and track the correction
            corrected_word = correction_dict[word]
            # Restore the original case of the word
            if original_case[index].istitle():
                corrected_word = corrected_word.capitalize()
            elif original_case[index].isupper():
                corrected_word = corrected_word.upper()
            new_word_list.append(corrected_word)
            corrections_tracker[word] = corrected_word

            # Extract context around the corrected word
            start_index = max(0, index - 10)
            end_index = min(len(clean_text_list), index + 11)
            context = clean_text_list[start_index:end_index]

            # Store the context with the correction
            corrections_tracker[word] = {
                'corrected_word': corrected_word,
                'context': ' '.join(context)
            }
        else:
            # If it doesn't, append the original word
            new_word_list.append(original_case[index])

    # Send data back to text
    new_corrected_text = ' '.join(new_word_list)

    return corrections_tracker, new_corrected_text

# Use the function and make two new columns to add to the dataframe
df['corrections_tracker'], df['ocr_corrected_text'] = zip(*tqdm(df['preprocessed_content'].apply(ocr_post_correction), desc="Applying OCR post-correction", colour='yellow'))

print ('Saving data')
# Save the df as a csv file
os.chdir(r'.\literature csv')
df.to_csv(f'{book_title}.csv', index=False)


# Save txt versions
os.chdir('..')
os.chdir(r'.\literature txt')

# Save text version without spell correction
text = ' '.join(df['preprocessed_content'].to_list())
with open(f'{book_title} without ocr correction.txt', 'w', encoding='utf-8') as f:
    f.write(text)

# Save text version with spell correction
# Join the corrected words back into a string
ocr_corrected_text = ' '.join(df['ocr_corrected_text'].to_list())
with open(f'{book_title} with ocr correction.txt', 'w', encoding='utf-8') as f:
    f.write(ocr_corrected_text)

########
# Save ocr corrections
print ('Save ocr corrections')
os.chdir('..')
os.chdir(r'.\ocr_correction_tracker')
obs_corrections_tracker = []
for i,j in enumerate(df['corrections_tracker'].to_list()):
    if j:
        observation = f"Index no.: {str(i)}. OCR observation: {str(j)}"
        observation = observation.replace('{', '').replace('}', '')
        obs_corrections_tracker.append(observation)

ocr_observations = '\n\n'.join(obs_corrections_tracker)

with open(f'{book_title} ocr corrections.txt', 'w', encoding='utf-8') as f:
    f.write(ocr_observations)

##############

print ('Script done.')

# change to main folder
os.chdir('..')

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Time taken to run the script: {elapsed_time:.6f} seconds")

import libraries
start time
Start processing "1860 A Residence in Jutland the Danish Isles and Copenhagen vol II".
Wait a few seconds.
Preprocess text
Starting ocr correction - loading correction rules
Ocr correction


Applying OCR post-correction: 100%|██████████| 1314/1314 [00:00<?, ?it/s]

Saving data
Save ocr corrections
Script done.
Time taken to run the script: 5.438003 seconds


In [9]:
#! pip install --upgrade langchain_community unstructured pypandoc

In [ ]:
# Interate a folder

In [ ]:
print ('import libraries')
# import libraries
from langchain_community.document_loaders import UnstructuredEPubLoader
from langchain.document_loaders import PyPDFLoader
import pandas as pd
import re
import os
import csv
import json
from tqdm import tqdm  # Import tqdm for progress tracking
import time

In [29]:
path_to_folder = r'C:\Users\lakj\Documents\GitHub\nordic travel literature\literature epub'
no_of_files = len(os.listdir(path_to_folder))
files = os.listdir(path_to_folder)

In [30]:
def input_is_folder(epub_file):
    # Save start time
    print ('start time')
    start_time = time.time()
    #######################
    
    # Move to epub folder
    os.chdir(r'.\literature epub')
    # load text from epub
    #epub_file = '1853 Visit to Iceland And the Scandinavian North.epub'
    book_title = epub_file[:-5]
    print (f'Start processing "{book_title}".\nWait a few seconds.')
    loader = UnstructuredEPubLoader(epub_file, mode="elements")
    epub_pages = loader.load()
    
    # Extract data from epub and send it to a dataframe
    content = []
    source = []
    element_no = []
    
    
    # add counter to element number
    count = 1 
    for i in epub_pages:
           
        # return the text content, source, and element no.
        pageContent = i.page_content
        pageSource = i.metadata['source'][:-5]
        elementNo = count
        
        content.append(pageContent)
        source.append(pageSource)
        element_no.append(elementNo)
    
        count += 1
        
    df = pd.DataFrame({'content': content, 'source': source, 'element_no': element_no})
    
    ######################
    print ('Preprocess text')
    # preprocess text
    def preproces_text(text_string):
        # Store text data in variable
        # Remove newlines, replace apostrophe s with s, remove quotation marks
        text_string_replace = text_string.replace(r' \xad\n', '').replace(r'\xad\n', '') \
            .replace(r'\xad', '').replace(r'\xad ', '') \
            .replace(r'-\n', '').replace('—', ' ').replace(r'\n', ' ') \
            .replace("'s", "s").replace('"', ' ').replace("'", " ") \
            .replace('.', ' . ').replace(' .', ' . ').replace(',', ' ,') \
            .replace('!', ' !').replace('?', ' ?')
    
        # scrub for multiple signs
        # but keep full stops, commas, exclamation, and question marks
        text_string_re = re.sub(r':|;||||\||\+|\"|‘|’|“|”|\’|…|\-|–|—|\$|&|\*|>|<|\/||||»|«', '', text_string_replace)
    
        # Remove numbers and words containing numbers
        # text_string_no_numbers = re.sub(r'\b\w*\d\w*\b', '', text_string_re)
        # Remove multiple whitespaces
        text_string_sub_additional_white_space = re.sub(r'\s+', ' ', text_string_re)
    
        return text_string_sub_additional_white_space
    
    df['preprocessed_content'] = df['content'].apply(lambda x : preproces_text(x))
    
    ################
    
    # go back to main folder
    os.chdir('..')
    
    
    print('Starting ocr correction - loading correction rules')
    # Read the correction rules from the file
    with open('Underwoods_CorrectionRules.txt', 'r', encoding='utf-8') as f:
        CorrectionRules_string = f.read()
    
    # Process the correction rules
    CorrectionRules_list = CorrectionRules_string.split('\n')
    list_of_CorrectionRules_list = [i.split('\t') for i in CorrectionRules_list]
    better_list_of_CorrectionRules = [i[0:2] for i in list_of_CorrectionRules_list]
    
    # Add additional pairs
    observations_to_add = [["Bruffels", 'Brussels'], ["fix", 'six'], ['Elsinore', 'Helsingør']]
    better_list_of_CorrectionRules.extend(observations_to_add)
    
    # Build correction dictionary
    correction_dict = dict(better_list_of_CorrectionRules)
    
    
    
    print('Ocr correction')
    
    def ocr_post_correction(text):
        # Prepare for correction
        ### Keep track of the conditions of the words (capital, title, upper,lower)
        tokens = text.split()
    
        # Track the original case of each word
        original_case = [i for i in tokens]
        clean_text_list = [i.lower() for i in tokens]
    
        # Initialize the list to store corrected words and a dictionary to track corrections
        new_word_list = []
        corrections_tracker = {}
    
        # Iterate through the clean_text_list word list
        for index, word in enumerate(clean_text_list):
            # Check if the word has a correction in the dictionary
            if word in correction_dict:
                # If it does, append the corrected word to the new list and track the correction
                corrected_word = correction_dict[word]
                # Restore the original case of the word
                if original_case[index].istitle():
                    corrected_word = corrected_word.capitalize()
                elif original_case[index].isupper():
                    corrected_word = corrected_word.upper()
                new_word_list.append(corrected_word)
                corrections_tracker[word] = corrected_word
    
                # Extract context around the corrected word
                start_index = max(0, index - 10)
                end_index = min(len(clean_text_list), index + 11)
                context = clean_text_list[start_index:end_index]
    
                # Store the context with the correction
                corrections_tracker[word] = {
                    'corrected_word': corrected_word,
                    'context': ' '.join(context)
                }
            else:
                # If it doesn't, append the original word
                new_word_list.append(original_case[index])
    
        # Send data back to text
        new_corrected_text = ' '.join(new_word_list)
    
        return corrections_tracker, new_corrected_text
    
    # Use the function and make two new columns to add to the dataframe
    df['corrections_tracker'], df['ocr_corrected_text'] = zip(*tqdm(df['preprocessed_content'].apply(ocr_post_correction), desc="Applying OCR post-correction", colour='yellow'))
    
    print ('Saving data')
    # Save the df as a csv file
    os.chdir(r'.\literature csv')
    df.to_csv(f'{book_title}.csv', index=False)
    
    
    # Save txt versions
    os.chdir('..')
    os.chdir(r'.\literature txt')
    
    # Save text version without spell correction
    text = ' '.join(df['preprocessed_content'].to_list())
    with open(f'{book_title} without ocr correction.txt', 'w', encoding='utf-8') as f:
        f.write(text)
    
    # Save text version with spell correction
    # Join the corrected words back into a string
    ocr_corrected_text = ' '.join(df['ocr_corrected_text'].to_list())
    with open(f'{book_title} with ocr correction.txt', 'w', encoding='utf-8') as f:
        f.write(ocr_corrected_text)
    
    ########
    # Save ocr corrections
    print ('Save ocr corrections')
    os.chdir('..')
    os.chdir(r'.\ocr_correction_tracker')
    obs_corrections_tracker = []
    for i,j in enumerate(df['corrections_tracker'].to_list()):
        if j:
            observation = f"Index no.: {str(i)}. OCR observation: {str(j)}"
            observation = observation.replace('{', '').replace('}', '')
            obs_corrections_tracker.append(observation)
    
    ocr_observations = '\n\n'.join(obs_corrections_tracker)
    
    with open(f'{book_title} ocr corrections.txt', 'w', encoding='utf-8') as f:
        f.write(ocr_observations)
    
    ##############
    
    print ('Script done.')
    
    # change to main folder
    os.chdir('..')
    
    # Record the end time
    end_time = time.time()
    
    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    
    # Print the elapsed time
    print(f"Time taken to run the script: {elapsed_time:.6f} seconds")

In [31]:
[input_is_folder(epub_file) for epub_file in files]

start time
Start processing "1773 Travels through Holland Flanders Germany Denmark Sweden Lapland Russia the Ukraine and Poland in the years 1768 1769 and 1770".
Wait a few seconds.
Preprocess text
Starting ocr correction - loading correction rules
Ocr correction


Applying OCR post-correction: 100%|██████████| 784/784 [00:00<?, ?it/s]

Saving data
Save ocr corrections
Script done.
Time taken to run the script: 2.448855 seconds
start time
Start processing "1795 A tour through some of the northern parts of Europe".
Wait a few seconds.


Preprocess text
Starting ocr correction - loading correction rules
Ocr correction


Applying OCR post-correction: 100%|██████████| 677/677 [00:00<?, ?it/s]

...

Saving data
Save ocr corrections
Script done.
Time taken to run the script: 1.163779 seconds
start time
Start processing "1963 Iceland Its Scenes and Its Sagas".
Wait a few seconds.


Preprocess text
Starting ocr correction - loading correction rules
Ocr correction


Applying OCR post-correction: 100%|██████████| 3773/3773 [00:00<00:00, 3773273.48it/s]

Saving data
Save ocr corrections
Script done.
Time taken to run the script: 6.587312 seconds


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]